** Neil Garrett, June 2018 **

# Start up commands/load relevant functions

In [3]:
# load required libraries
using Distributed

# # set everything up
parallel = true # Run on multiple CPUs. 

# this activates the multiprocessing threads
if (parallel)
	# only run this once
    addprocs(4)
end

# load required libraries

@everywhere using DataFrames
#using DataArrays
@everywhere using ForwardDiff
@everywhere using PyCall
@everywhere using Distributions
@everywhere using PyPlot
@everywhere using CSV
@everywhere using SpecialFunctions
@everywhere using SharedArrays
@everywhere using LinearAlgebra

@everywhere PyCall.@pyimport scipy.optimize as so

# this is the code for the actual fitting routines
@everywhere include("em.jl")
@everywhere include("common.jl")
@everywhere include("likfuns.jl")

# this is generates starting matricies for betas, sigmas etc to feed into model
@everywhere include("genVars.jl")

┌ Warning: Deprecated syntax `(covvar < 0) ? NaN:` at /Users/neil/GitHubRepo/Projects/PreySelection/v103/models/supplementary/model_asymmetry_learn_options/em.jl:288.
│ Use `(covvar < 0) ? NaN :` instead.
└ @ nothing /Users/neil/GitHubRepo/Projects/PreySelection/v103/models/supplementary/model_asymmetry_learn_options/em.jl:288
┌ Warning: Deprecated syntax `(covvar < 0) ? NaN :sqrt` at /Users/neil/GitHubRepo/Projects/PreySelection/v103/models/supplementary/model_asymmetry_learn_options/em.jl:288.
│ Use `(covvar < 0) ? NaN : sqrt` instead.
└ @ nothing /Users/neil/GitHubRepo/Projects/PreySelection/v103/models/supplementary/model_asymmetry_learn_options/em.jl:288
┌ Warning: Deprecated syntax `(diag(covmtx)[i] .< 0) ? NaN:` at /Users/neil/GitHubRepo/Projects/PreySelection/v103/models/supplementary/model_asymmetry_learn_options/em.jl:299.
│ Use `(diag(covmtx)[i] .< 0) ? NaN :` instead.
└ @ nothing /Users/neil/GitHubRepo/Projects/PreySelection/v103/models/supplementary/model_asymmetry_learn_o

┌ Warning: Deprecated syntax `parametric method syntax flatten{T}(a::Array{T})` around /Users/neil/GitHubRepo/Projects/PreySelection/v103/models/supplementary/model_asymmetry_learn_options/common.jl:67.
│ Use `flatten(a::Array{T}) where T` instead.
└ @ ~/GitHubRepo/Projects/PreySelection/v103/models/supplementary/model_asymmetry_learn_options/common.jl:67
┌ Warning: Deprecated syntax `parametric method syntax flatten{T}(a::Array{T, 1})` around /Users/neil/GitHubRepo/Projects/PreySelection/v103/models/supplementary/model_asymmetry_learn_options/common.jl:66.
│ Use `flatten(a::Array{T, 1}) where T` instead.
└ @ ~/GitHubRepo/Projects/PreySelection/v103/models/supplementary/model_asymmetry_learn_options/common.jl:66
┌ Warning: Deprecated syntax `parametric method syntax flatten{T}(a::Array{T})` around /Users/neil/GitHubRepo/Projects/PreySelection/v103/models/supplementary/model_asymmetry_learn_options/common.jl:67.
│ Use `flatten(a::Array{T}) where T` instead.
└ @ ~/GitHubRepo/Projects/Pre

# Data read and process

### Read in trial by trial data

In [2]:
#read in csv file of the data
#trial by trial data: note will include force trials and missed responses
df = readtable("/Users/neil/GitHubRepo/Projects/PreySelection/v103/data/trialdata_103_processed.csv")

#display header
head(df)


┌ Warning: readtable is deprecated, use CSV.read from the CSV package instead
│   caller = top-level scope at In[2]:1
└ @ Core In[2]:1


,subj,trial_index_actual,block,stimulus,stim_rank,reward_percent,delay_s,profitability,stim_left_right,key_press,approach_avoid,rt,rt_z,force_trial,missed,order_condition,exclude,exclude_reason
,Int64⍰,Int64⍰,Int64⍰,String⍰,Int64⍰,Int64⍰,Int64⍰,Float64⍰,String⍰,Int64⍰,Float64⍰,Float64⍰,Float64⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰,String⍰
1,1,0,1,../static/images/invador2.png,4,20,8,2.5,left,74,-1.0,1172.0,1.475,0,0,2,0,do not exclude
2,1,1,1,../static/images/invador3.png,1,80,2,40.0,right,74,1.0,1046.0,0.846311,1,0,2,0,do not exclude
3,1,2,1,../static/images/invador1.png,3,80,8,10.0,left,70,1.0,743.0,-0.665547,0,0,2,0,do not exclude
4,1,3,1,../static/images/invador2.png,4,20,8,2.5,right,74,NaN,NaN,NaN,1,1,2,0,do not exclude
5,1,4,1,../static/images/invador2.png,4,20,8,2.5,left,70,1.0,858.0,-0.0917396,0,0,2,0,do not exclude
6,1,5,1,../static/images/invador4.png,2,20,2,10.0,left,70,1.0,818.0,-0.291325,0,0,2,0,do not exclude


### Append data with the column "sub" 


In [3]:
#this is just a replica of the existing column sub_no but think em looks for "sub" specifically
df[:sub] = df[:subj];


### Get rid of excluded subs

In [4]:
df = df[df[:exclude].==0,:];


### Convert approach avoid to 2s and 1s , missed as 0. Then convert to integers (necessary to use as an index)

In [5]:
#convert approach_avoid to 1s (avoid) and 2s (approach)
df[df[:approach_avoid].==1,:approach_avoid] = 2
df[df[:approach_avoid].==-1,:approach_avoid] = 1

index_NaN = find(isnan.(df[:approach_avoid]))
df[index_NaN, :approach_avoid] = 0

df[:approach_avoid] = convert(Vector{Integer}, df[:approach_avoid])

head(df)


┌ Warning: `find(A::AbstractVector)` is deprecated, use `findall(A)` instead.
│   caller = top-level scope at In[5]:4
└ @ Core In[5]:4


,subj,trial_index_actual,block,stimulus,stim_rank,reward_percent,delay_s,profitability,stim_left_right,key_press,approach_avoid,rt,rt_z,force_trial,missed,order_condition,exclude,exclude_reason,sub
,Int64⍰,Int64⍰,Int64⍰,String⍰,Int64⍰,Int64⍰,Int64⍰,Float64⍰,String⍰,Int64⍰,Integer,Float64⍰,Float64⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰,String⍰,Int64⍰
1,1,0,1,../static/images/invador2.png,4,20,8,2.5,left,74,1,1172.0,1.475,0,0,2,0,do not exclude,1
2,1,1,1,../static/images/invador3.png,1,80,2,40.0,right,74,2,1046.0,0.846311,1,0,2,0,do not exclude,1
3,1,2,1,../static/images/invador1.png,3,80,8,10.0,left,70,2,743.0,-0.665547,0,0,2,0,do not exclude,1
4,1,3,1,../static/images/invador2.png,4,20,8,2.5,right,74,0,NaN,NaN,1,1,2,0,do not exclude,1
5,1,4,1,../static/images/invador2.png,4,20,8,2.5,left,70,2,858.0,-0.0917396,0,0,2,0,do not exclude,1
6,1,5,1,../static/images/invador4.png,2,20,2,10.0,left,70,2,818.0,-0.291325,0,0,2,0,do not exclude,1


### Read in summary stats

In [6]:
summary_stats = readtable("/Users/neil/GitHubRepo/Projects/PreySelection/v103/data/subdata_103.csv")
head(summary_stats)

┌ Warning: readtable is deprecated, use CSV.read from the CSV package instead
│   caller = top-level scope at In[6]:1
└ @ Core In[6]:1


,MturkID,AssignID,sub_no,age,gender,n_trials,n_force_wrong,n_missed,bonus_payment,percent_accept_A1,percent_accept_A2,percent_accept_A3,percent_accept_A4,percent_accept_A2_A3,percent_accept_B1,percent_accept_B2,percent_accept_B3,percent_accept_B4,percent_accept_B2_B3,percent_accept_B1_min_A1,percent_accept_B2_min_A2,percent_accept_B3_min_A3,percent_accept_B4_min_A4,percent_accept_B2_B3_min_A2_A3,percent_accept_AB1,percent_accept_AB2,percent_accept_AB3,percent_accept_AB4,percent_accept_AB2_AB3,exclude,exclude_reason,order_condition,comment
,String⍰,String⍰,Int64⍰,Int64⍰,String⍰,Int64⍰,Int64⍰,Int64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Int64⍰,String⍰,Int64⍰,String⍰
1,A3V1WO6CU1EQA1,3TXWC2NHNZQV07CW65J93JAODMK9SI,1,23,M,434,4,4,3.8,0.989362,0.0,0.15,0.037037,0.075,0.947368,0.592593,0.761905,0.134021,0.666667,-0.0419933,0.592593,0.611905,0.0969836,0.591667,0.982301,0.340426,0.463415,0.112903,0.397727,0,do not exclude,2,"It would be great to have a feature that shows your current amount of fuel collected. Otherwise, great hit!"
2,A2VRDE2FHCBMF8,3QHK8ZVMIMIRKNW3KG04I3RJ9SFBLG,2,38,m,437,57,29,3.6,0.988235,0.85,0.0,0.0,0.395349,0.904762,0.958333,0.0357143,0.0215054,0.461538,-0.0834734,0.108333,0.0357143,0.0215054,0.0661896,0.971698,0.909091,0.0196078,0.0183486,0.431579,1,>19 missed,2,"It has been funny, thank you! When I missed too many * I decided to change my strategy ad I aimed just for the grey and the green invaders despite the position of other *."
3,AHPOUVK6YUTK1,3A4NIXBJ76ZN3E00B5QDWJ2DLRJLMF,3,32,f,480,3,2,3.8,0.980769,0.96,0.0,0.0833333,0.470588,1.0,1.0,0.0,0.0,0.464286,0.0192308,0.04,0.0,-0.0833333,-0.00630252,0.985294,0.980392,0.0,0.0147059,0.46729,0,do not exclude,1,"That was fun, but my arm hurts from sitting in the same position for too long!"
4,A150GMV1YQWWB3,3TE22NPXPBCB1G49Z9R0O8J2DE344A,4,37,m,409,2,5,3.9,0.98913,0.9,0.818182,0.0416667,0.857143,0.904762,0.52381,1.0,0.021978,0.761905,-0.0843685,-0.37619,0.181818,-0.0196886,-0.0952381,0.973451,0.707317,0.906977,0.026087,0.809524,0,do not exclude,2,
5,A5EU1AQJNC7F2,3F1567XTNW5JNRXJE1U48JVNVYZ9QT,5,28,F,386,13,28,3.5,0.933333,0.473684,0.347826,0.1,0.404762,0.947368,0.764706,0.736842,0.0789474,0.75,0.0140351,0.291022,0.389016,-0.0210526,0.345238,0.93617,0.611111,0.52381,0.0833333,0.564103,1,>19 missed,2,none
6,AHDBHMH3AY1V2,3FTOP5WARFOK5A5TJFR2I7I8EZDJ00,6,31,m,421,1,3,3.9,1.0,0.833333,0.0454545,0.136364,0.456522,1.0,0.875,1.0,0.043956,0.948718,0.0,0.0416667,0.954545,-0.0924076,0.492196,1.0,0.85,0.533333,0.0619469,0.682353,0,do not exclude,2,


### Get rid of excluded subs

In [7]:
summary_stats = summary_stats[summary_stats[:exclude].==0,:];


### Get rid of Mturk ID etc (first 3 columns)

In [8]:
summary_stats = summary_stats[:,4:end];

┌ Warning: indexing with colon as row will create a copy in the future use df[col_inds] to get the columns without copying
│   caller = top-level scope at In[8]:1
└ @ Core In[8]:1


# Asymmetry Model Learn Options

This model comprises: 

1. An intercept which reflects degree of bias to reject.

2. A beta (termperature parameter) which controls sensitivity to the difference between the options (0 = pick 50/50. Higher it is, the more sensative subs are tothe different options (more step functionesque). <br>

3. Two learning rates: one for appetative component (reward), one for aversive (delay) for average reward rate

4. Seperate learning rate for learning the value of the options (no longer assume they know this)

Uses Q learned average to predict choice

Initalise Qaverage in model at the arithmetic average over all subs over both sessions

In [11]:
@everywhere function model_asymmetry_learn_options(params, data)
     
    #model parameters
    intercept = params[1]
    beta = params[2]
    lr_environment_reward = 0.5 .+ 0.5.*erf(params[3]/sqrt(2))
    lr_environment_delay = 0.5 .+ 0.5.*erf(params[4]/sqrt(2))
    lr_options = 0.5 .+ 0.5.*erf(params[5]/sqrt(2))
   
    
    #initalise various variables
    delay_sum = zeros(typeof(beta),1)
    reward_sum = zeros(typeof(beta),1)
    Q_arithmetic = zeros(typeof(beta),1) # stores actual (arithmetic) average reward rate
    opp_cost_arithmetic = zeros(typeof(beta),1) # stores actual (arithmetic) opp cost
    Q_estimate = zeros(typeof(beta),1) .+ 8.22 # stores estimated global reward rate
    opp_cost_estimate = zeros(typeof(beta),1) # stores estimated opp cost
    
    Qd = zeros(typeof(beta),2) # decision variable; 1st element is the opp cost of accepting (or value of rejecting), 2nd element is just the reward of the option (value of accepting)

    # maintains estimate of the reward for each option (options indexed by rank: 1-4). 
    Q_options_reward = zeros(typeof(beta),4)
    Q_options_delay = zeros(typeof(beta),4)
    
    lik = 0 #likelihood

    #these store new trial by trial values (e.g. Q estimate on each trial etc.)
    reward_sum_store = [];
    delay_sum_store = [];
    Q_arithmetic_store = [];
    opp_cost_arithmetic_store  = []; 
    Q_estimate_store = [];
    opp_cost_estimate_store = [];

    #extract various variables from the dataframe
    reward = data[:reward_percent]
    delay = data[:delay_s]
    force = data[:force_trial]  
    missed = data[:missed] #missed responses 
    c = data[:approach_avoid] #choice
    option_rank = data[:stim_rank] #option rank 

    for i = 1:length(c)
        
            #option presented in current trial
            option_index = option_rank[i]
        
            # 2 seconds without reward on each trial regadless of accept/reject
            delay_sum .+= 2;
         
            # calculate current (arithmetic) reward per second from number of seconds elapsed and reward accured
            Q_arithmetic = reward_sum./delay_sum
            opp_cost_arithmetic = Q_arithmetic*delay[i]
           
            # decrease estimate of global reward rate for encounter time (2seconds)
            Q_estimate = (1-lr_environment_delay) * Q_estimate .+ 0
            Q_estimate = (1-lr_environment_delay) * Q_estimate .+ 0
        
            #calculate estimate of opportunity cost given estimate of reward rate and delay incurred by option 
            opp_cost_estimate = Q_estimate*Q_options_delay[option_index]
        
            #add trial by trial values 
            append!(reward_sum_store, reward_sum)
            append!(delay_sum_store, delay_sum)
            append!(Q_arithmetic_store, Q_arithmetic)
            append!(opp_cost_arithmetic_store, opp_cost_arithmetic)
            append!(Q_estimate_store, Q_estimate)
            append!(opp_cost_estimate_store, opp_cost_estimate)
        
            # if not a force trial predict choice based on current values
            if ((force[i]<1) & (missed[i]<1))
                        
                # decision variable - the estimate of opportunity cost ("reward" of rejecting) versus 
                # reward of the current option (if accepted)
                Qd = [intercept, 0] .+ [beta.*opp_cost_estimate[1], beta.*Q_options_reward[option_index]]

                # increment likelihood
                lik += Qd[c[i]] - log(sum(exp.(Qd)))
            
            end
            
            #incur 8second time out for missed response
            if (missed[i]==1)
                
                delay_sum .+= 8
            
                for j = 1:8
                
                     Q_estimate = (1-lr_environment_delay) * Q_estimate .+ 0

                end
            
            end
        
            # regardless of whether a force trial or not, 
            # if accept the option, Q_estimate updates and there is a delay incurred
            if ((c[i] == 2) & (missed[i]==0))
                
                delay_sum .+= delay[i]
                reward_sum .+= reward[i]
            
                for j = 1:delay[i]
                
                    Q_estimate = (1-lr_environment_delay) * Q_estimate .+ 0
                
                end
            
                    Q_estimate = (1-lr_environment_reward) * Q_estimate .+ lr_environment_reward*reward[i]
            
                Q_options_reward[option_index] = (1-lr_options)*Q_options_reward[option_index] .+ lr_options*reward[i]
                Q_options_delay[option_index] = (1-lr_options)*Q_options_delay[option_index] .+ lr_options*delay[i]
                
            end
    
    end
    
    # compile trial by trial values here
    trial_data = DataFrame(reward_sum = reward_sum_store,
            delay_sum = delay_sum_store,
            Q_arithmetic = Q_arithmetic_store,
            opp_cost_arithmetic = opp_cost_arithmetic_store,
            Q_estimate = Q_estimate_store,
            opp_cost_estimate = opp_cost_estimate_store)
    
    # here if running em you can only return the likelihood
    return -lik
    
    # but if you run in order to extract trials, subs etc then want to return this
    #return (-lik, trial_data)
    
end


# Parameter optimisiation

### Run model for one subject

aids debugging

In [12]:
# initialize parameter structures
(df, subs, X, betas, sigma) = genVars(df, 5);

# run model for sub 1
model_asymmetry_learn_options(betas,df[df[:sub].==subs[1],:])


225.2728336819827

### Run em to get best fit parameters for each subject

In [13]:
# initialized parameter structures (again)
# note that some of the variables (e.g. betas, sigma) are entered and returned by em function 
(df, subs, X, betas, sigma) = genVars(df, 5);

# run for full learner
# x contains the parameters for each subject (note not the same as variable X)
# l and h are per-subject likelihood and hessians
@time (betas, sigma, x, l, h) = em(df, subs, X, betas, sigma, model_asymmetry_learn_options; emtol=1e-3, parallel=true, full=true, quiet=false);



iter: 32
betas: [-1.8, 0.09, -2.56, -2.95, 0.23]
sigma: [4.58 0.01 -0.4 0.2 -1.19; 0.01 0.0 -0.01 -0.0 -0.01; -0.4 -0.01 0.44 0.31 0.21; 0.2 -0.0 0.31 0.33 0.06; -1.19 -0.01 0.21 0.06 1.24]
change: [-0.0, 1.2e-5, -0.0, -1.0e-6, 4.8e-5, 2.0e-6, 3.9e-5, -6.0e-6, 3.1e-5, -7.0e-6, 3.6e-5, -1.1e-5, -0.000338, -0.000259, 1.4e-5, 1.2e-5, 3.7e-5, 1.7e-5, 0.000227, 5.6e-5]
max: 0.000227
164.180445 seconds (43.49 M allocations: 1.662 GiB, 0.40% gc time)


### Generate Model Statistics 
(IAIC, IBIC, LOOCV)

In [14]:
## model selection/comparison/scoring

# laplace approximation to the aggregate log marginal likelihood of the whole dataset
# marginalized over the individual params

aggll = lml(x,l,h)

# to compare this between models you need to correct for the group-level free parameters
# either aic or bic

aggll_ibic = ibic(x,l,h,betas,sigma,nrow(df))
aggll_iaic = iaic(x,l,h,betas,sigma)

# or you can compute unbiased per subject marginal likelihoods via subject-level cross validation
# you can do paired t tests on these between models
# these are also appropriate for SPM_BMS etc

# takes ages so comment in when want to run, otherwise just use IAIC above
liks = loocv(df, subs, x, X, betas, sigma, model_asymmetry_learn_options; emtol=1e-3, parallel=true, full=true)
#aggll_loo = sum(liks)

#println("\n\nraw nll:  $aggll\nibic nll: $aggll_ibic\niaic nll: $aggll_iaic\nloo nll:  $aggll_loo")
#println("\n\nraw nll:  $aggll\nibic nll: $aggll_ibic\niaic nll:")
#print(aggll_iaic)


Subject: 1..2..3..4..5..6..7..8..9..10..11..12..13..14..15..16..17..18..19..20..21..22..23..24..25..26..27..28..29..30..31..32..33..34..35..36..37..38..39..40..

40-element Array{Float64,1}:
 105.17597582832133 
  30.374000056542812
  73.9450348243689  
  68.84728770343057 
  83.27692019115374 
  82.53203577001136 
  30.31638726896194 
  58.143322032764786
  41.01649778229009 
  71.2082641850933  
  58.83034758465124 
  24.88119710097701 
 118.42196928296562 
   ⋮                
  64.95063079704879 
  88.53411427716192 
  80.61482297090673 
  52.521056368107914
 100.2780627009526  
  68.47293957150933 
  53.28454865666784 
  66.25265161292873 
 127.89801127485227 
  65.58760464276578 
  91.64703335078723 
  27.658485794392654

### Write loocv scores to csv file

(if you have run loocv above)

In [15]:
# put loocv scores into dataframe
loocv_scores = DataFrame(sub = subs,
liks = vec(liks));


#### save LOOCV to csv file


In [16]:
CSV.write("loocv_scores.csv", DataFrame(loocv_scores))


┌ Warning: In the future DataFrame constructor called with a `DataFrame` argument will return a copy. Use `convert(DataFrame, df)` to avoid copying if `df` is a `DataFrame`.
│   caller = top-level scope at In[16]:1
└ @ Core In[16]:1


"loocv_scores.csv"

#### add to summary stats to LOOCV as well

In [17]:
summary_stats = [summary_stats loocv_scores];

### Calculate and write p values, std error and covariance

In [18]:
# standard errors on the subject-level means, based on an asymptotic Gaussian approx 
# (these may be inflated esp for small n)
(standarderrors, pvalues, covmtx) = emerrors(df,subs,x,X,h,betas,sigma,model_asymmetry_learn_options);


  likely near /Users/neil/.julia/packages/IJulia/DL02A/src/kernel.jl:41
  likely near /Users/neil/.julia/packages/IJulia/DL02A/src/kernel.jl:41
  likely near /Users/neil/.julia/packages/IJulia/DL02A/src/kernel.jl:41
in #53 at none
  likely near /Users/neil/.julia/packages/IJulia/DL02A/src/kernel.jl:41
  likely near /Users/neil/.julia/packages/IJulia/DL02A/src/kernel.jl:41
  likely near /Users/neil/.julia/packages/IJulia/DL02A/src/kernel.jl:41
in #53 at none
┌ Warning: `ccdf(d::UnivariateDistribution, X::AbstractArray)` is deprecated, use `ccdf.(d, X)` instead.
│   caller = emerrors(::DataFrame, ::Array{Union{Missing, Int64},1}, ::SharedArray{Float64,2}, ::Array{Float64,3}, ::SharedArray{Float64,3}, ::Array{Float64,1}, ::Array{Float64,2}, ::Function) at em.jl:300
└ @ Main /Users/neil/GitHubRepo/Projects/PreySelection/v103/models/supplementary/model_asymmetry_learn_options/em.jl:300


In [19]:
model_stats = DataFrame(stderror = vec(standarderrors),
pvalues = vec(pvalues),
covmtx_1 = vec(covmtx[:,1]),
covmtx_2 = vec(covmtx[:,2]),
covmtx_3 = vec(covmtx[:,3]),
covmtx_4 = vec(covmtx[:,4]),
covmtx_5 = vec(covmtx[:,5]));

# save model stats to csv file
CSV.write("model_stats.csv", DataFrame(model_stats));

┌ Warning: In the future DataFrame constructor called with a `DataFrame` argument will return a copy. Use `convert(DataFrame, df)` to avoid copying if `df` is a `DataFrame`.
│   caller = top-level scope at In[19]:8
└ @ Core In[19]:8


In [20]:
print(standarderrors)


[0.351078, 0.00557931, 0.109808, 0.10232, 0.220723]

In [21]:
print(pvalues)


[2.96909e-7, 2.52355e-57, 3.60892e-120, 3.75563e-183, 0.305061]

In [22]:
print(covmtx)


[0.123256 0.000177681 -0.0106329 0.00512219 -0.0319179; 0.000177681 3.11287e-5 -0.000249284 -5.92755e-5 -0.00023785; -0.0106329 -0.000249284 0.0120577 0.00904528 0.00526887; 0.00512219 -5.92755e-5 0.00904528 0.0104694 0.00129002; -0.0319179 -0.00023785 0.00526887 0.00129002 0.0487188]

### Write per subject model parameters to csv files

In [23]:
# put parameters into variable d
d=x';

# now put parameters into dataframe
params = DataFrame(sub = subs,
intercept = vec(d[:,1]), 
beta = vec(d[:,2]),
learning_rate_environment_reward_raw = vec(d[:,3]),
learning_rate_environment_reward_transformed = vec(0.5 .+ 0.5*erf.(d[:,3]/sqrt(2))),
learning_rate_environment_delay_raw = vec(d[:,4]),
learning_rate_environment_delay_transformed = vec(0.5 .+ 0.5*erf.(d[:,4]/sqrt(2))),
learning_rate_options_raw = vec(d[:,5]),
learning_rate_options_transformed = vec(0.5 .+ 0.5*erf.(d[:,5]/sqrt(2))));

In [24]:
CSV.write("subject_params.csv", DataFrame(params))


┌ Warning: In the future DataFrame constructor called with a `DataFrame` argument will return a copy. Use `convert(DataFrame, df)` to avoid copying if `df` is a `DataFrame`.
│   caller = top-level scope at In[24]:1
└ @ Core In[24]:1


"subject_params.csv"

### Test difference in learning rates

In [4]:
# if you already have best fit parameters saved, can read in here (rather than running model to find)
params = CSV.read("subject_params.csv")

,sub,intercept,beta,learning_rate_environment_reward_raw,learning_rate_environment_reward_transformed,learning_rate_environment_delay_raw,learning_rate_environment_delay_transformed,learning_rate_options_raw,learning_rate_options_transformed
,Int64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰
1,1,0.174478,0.0734811,-1.92948,0.0268359,-2.09015,0.0183023,-0.949774,0.171114
2,3,-5.98517,0.0420079,-1.69662,0.0448843,-3.00412,0.00133177,1.29346,0.902074
3,4,-0.103493,0.105737,-3.5085,0.000225319,-3.46941,0.000260805,-0.822035,0.205528
4,6,-1.97819,0.0865272,-2.12014,0.0169971,-2.36357,0.00904989,0.419748,0.662665
5,8,1.46089,0.0921156,-2.06339,0.0195377,-1.98504,0.0235699,0.480433,0.68454
6,9,0.268886,0.0844414,-3.13601,0.000856317,-3.32819,0.000437063,-0.542262,0.293819
7,10,-3.47929,0.125164,-2.76586,0.00283862,-3.1758,0.000747118,0.924281,0.82233
8,12,-3.22742,0.0703459,-1.74428,0.0405548,-2.28356,0.0111988,1.55558,0.940096
9,13,0.59332,0.163037,-3.65239,0.000129905,-3.24685,0.000583439,-0.520887,0.301223


In [5]:
# if you already have best fit parameters saved, can read in here (rather than running model to find)
model_stats = CSV.read("model_stats.csv")

,stderror,pvalues,covmtx_1,covmtx_2,covmtx_3,covmtx_4,covmtx_5
,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰
1,0.351078,2.96909e-7,0.123256,0.000177681,-0.0106329,0.00512219,-0.0319179
2,0.00557931,2.52355e-57,0.000177681,3.11287e-5,-0.000249284,-5.92755e-5,-0.00023785
3,0.109808,3.60892e-120,-0.0106329,-0.000249284,0.0120577,0.00904528,0.00526887
4,0.10232,3.75563e-183,0.00512219,-5.92755e-5,0.00904528,0.0104694,0.00129002
5,0.220723,0.305061,-0.0319179,-0.00023785,0.00526887,0.00129002,0.0487188


In [23]:
covmtx = convert(Array, model_stats[:,3:7])

┌ Warning: indexing with colon as row will create a copy in the future use df[col_inds] to get the columns without copying
│   caller = top-level scope at In[23]:1
└ @ Core In[23]:1


5×5 Array{Union{Missing, Float64},2}:
  0.123256      0.000177681  -0.0106329     0.00512219  -0.0319179 
  0.000177681   3.11287e-5   -0.000249284  -5.92755e-5  -0.00023785
 -0.0106329    -0.000249284   0.0120577     0.00904528   0.00526887
  0.00512219   -5.92755e-5    0.00904528    0.0104694    0.00129002
 -0.0319179    -0.00023785    0.00526887    0.00129002   0.0487188 

In [28]:
# require contrast vector to test the difference in the learning parameters (from the covariance matrix)
con_vec = [0, 0, 1, -1, 0]

#calculate standard error
std_error = sqrt(con_vec'*covmtx*con_vec)

#difference in (raw) learning rates
learning_parm_diff = mean(params[:learning_rate_environment_reward_raw]) - mean(params[:learning_rate_environment_delay_raw])

#now can derive tstatistic
z_stat = learning_parm_diff/std_error

#print these to output
println("learning_parm_diff: ", learning_parm_diff);
println("std_error: ", std_error);
println("zstat: ", z_stat);

learning_parm_diff: 0.39351654926940194
std_error: 0.06660788088253787
zstat: 5.907957798017374
